For google colab environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/Othercomputers/ASUS Main/MLCP/03_Classifier/LLaMA-Factory/"

/content/drive/Othercomputers/ASUS Main/MLCP/03_Classifier/LLaMA-Factory


In [ ]:
pip install --upgrade huggingface_hub


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `LLM_TRAIN` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLM_TRAI

In [ ]:
!python -m pip install --upgrade pip
!pip install -e .
!pip install rouge_chinese

Obtaining file:///content/drive/Othercomputers/ASUS%20Main/MLCP/03_Classifier/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.3.dev0-0.editable-py3-none-any.whl size=26743 sha256=1633d8787d961e0e69b2a4ea3fa5a0fec38922a85c9be854185804d7d7a6450b
  Stored in directory: /tmp/pip-ephem-wheel-cache-f1oszegr/wheels/56/c9/c3/6f1bef4ffcb5ed14069ff7f945804c6a0c82d3e76e40c30f96
Successfully built llamafactory
  Attempting uninstall: llamafactory
    Found existing installation: llamafactory 0.9.3.dev0
    Uninstalling llamafactory-0.9.3.dev0:
      Successfully uninstalled llamafactory-0.9.3.dev0


Start code here

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["GEMMA_ATTENTION_IMPL"]="eager"


In [ ]:
!python src/webui.py

2025-06-14 12:01:50.764593: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-14 12:01:50.782542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749902510.804409   21670 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749902510.811087   21670 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-14 12:01:50.832882: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
import ast
import math
from typing import List, Sequence, Union

TOP_5_OUR_DATASET_TAGS = "['data structures', 'greedy', 'math', 'implementation', 'dp']"
TOP_10_OUR_DATASET_TAGS = "['greedy', 'dp', 'graphs', 'brute force', 'math', 'constructive algorithms', 'sortings', 'implementation', 'binary search', 'data structures']"
TOP_20_OUR_DATASET_TAGS = "['implementation', 'binary search', 'math', 'number theory', 'greedy', 'graphs', 'data structures', 'geometry', 'sortings', 'dp', 'brute force', 'combinatorics', 'dfs and similar', 'constructive algorithms', 'trees', 'strings', 'two pointers', 'dsu', 'bitmasks', 'divide and conquer']"

# ALWAYS UPDATE TAG_LIST
TAG_LIST = TOP_5_OUR_DATASET_TAGS

def _to_list(value: Union[str, Sequence[str], None]) -> List[str]:
    """Convert any tag representation → a list of clean tag strings."""
    if value is None:
        return []
    # Pandas NaN is float and math.isnan() is True
    if isinstance(value, float) and math.isnan(value):
        return []
    # Already a Python list
    if isinstance(value, (list, tuple)):
        return [str(t).strip() for t in value if str(t).strip()]
    # Make it a trimmed string and strip line-breaks
    value = str(value).strip().replace("\n", "")
    if value == "":
        return []
    # Case 1: looks like JSON / Python list
    if value.startswith("[") and value.endswith("]"):
        try:
            return [str(t).strip() for t in ast.literal_eval(value)]
        except Exception:
            pass  # fall through to comma split
    # Case 2: plain comma-separated line
    return [t.strip() for t in value.split(",") if t.strip()]

def create_binary_vector(tag_like, unique_tags=TAG_LIST) -> List[int]:
    """Return multi-hot vector the length of `unique_tags`."""
    # ensure unique_tags is a list
    if isinstance(unique_tags, str):
        unique_tags = ast.literal_eval(unique_tags)
    tags = _to_list(tag_like)
    vector = [0] * len(unique_tags)
    for tag in tags:
        if tag in unique_tags:
            vector[unique_tags.index(tag)] = 1
    return vector


In [ ]:

from sklearn.metrics import f1_score, roc_auc_score
import pandas as pd
import json
import os

generated_results = []
with open("saves/Llama-3-8B/lora/eval_2025-06-14-12-02-52_top_5_2025/generated_predictions.jsonl", "r") as file:
    for line in file:
        generated_results.append(json.loads(line))

truths = []
predictions = []

# Iterate through the test dataset and generated results
for data in generated_results:
    # Extract the relevant fields from the test dataset
    truths.append(data["label"])
    predictions.append(data["predict"])


merged_df = pd.DataFrame({
    'truths': truths,
    'predictions': predictions
})

merged_df['truths'] = merged_df['truths'].apply(lambda x: create_binary_vector(x, TAG_LIST))
merged_df['predictions'] = merged_df['predictions'].apply(lambda x: create_binary_vector(x, TAG_LIST))

# Calculate F1 macro score
f1_macro = f1_score(merged_df['truths'].tolist(), merged_df['predictions'].tolist(), average='macro')

# Calculate AUROC
auroc = roc_auc_score(merged_df['truths'].tolist(), merged_df['predictions'].tolist(), average='macro')

print(f"F1 Macro Score: {f1_macro}")
print(f"AUROC: {auroc}")

F1 Macro Score: 0.6718207282913166
AUROC: 0.7875128635983899
